Load data from shared folder

In [1]:
# Load Libraries
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
from matplotlib.path import Path
from multiprocess import Pool
import networkx as nx

%matplotlib inline

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *
from distributed_utilities import download_from_s3_to_ec2

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node
rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


No vtk
Child returned 0
Child returned 1
0.40 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.36 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


In [2]:
# Setting Parameters
n_radial = 4
# radial_bins = np.logspace(0, 2, 10, base=10)
radial_bins = np.linspace(0, 100, n_radial+1)

n_angular = 8
angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

n_orientation_bins = 10
orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

size_histogram_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

n_edge_length_bins = 10
edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]

n_edge_direction_bins = 10
edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)

# Patch paramters
patch_size = 224
half_size = patch_size/2
patch_spacing = 56

#

In [3]:
stack = 'MD589'
sec = 200

In [4]:

# Load single cell features from shared folder. /shared/CSHL_*
try:
    cell_sizes = load_cell_data('cellSizes', stack=stack, sec=sec, ext='bp')
except:
    print('fail to read')


rm -rf /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N36-2015.07.30-18.28.52_MD589_2_0107/MD589-N36-2015.07.30-18.28.52_MD589_2_0107_cellSizes.bp && mkdir -p /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N36-2015.07.30-18.28.52_MD589_2_0107
aws s3 cp s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-N36-2015.07.30-18.28.52_MD589_2_0107/MD589-N36-2015.07.30-18.28.52_MD589_2_0107_cellSizes.bp /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N36-2015.07.30-18.28.52_MD589_2_0107/MD589-N36-2015.07.30-18.28.52_MD589_2_0107_cellSizes.bp


Child returned 0


fail to read


Child returned 1
0.36 seconds.


In [5]:
grid_spec = (patch_size, patch_spacing,) + metadata_cache['image_shape'][stack]
grid_locations = grid_parameters_to_sample_locations(grid_spec)

first_section, last_section = metadata_cache['section_limits'][stack]

In [6]:
is_invalid(sec=sec, stack=stack)

False

In [7]:
n_patch_shape = np.array(metadata_cache['image_shape']['MD589'])/patch_size * 4
n_patch_shape[0] * n_p

NameError: name 'n_p' is not defined

In [8]:
metadata_cache['sections_to_filenames'][stack][sec]

'MD589-N36-2015.07.30-18.28.52_MD589_2_0107'

In [9]:
mask_tb = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec)

File does not exist: /shared/CSHL_data_processed/MD589/MD589_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_masks_cropped/MD589-N36-2015.07.30-18.28.52_MD589_2_0107_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_mask_cropped.png


IOError: [Errno 2] No such file or directory: '/shared/CSHL_data_processed/MD589/MD589_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_masks_cropped/MD589-N36-2015.07.30-18.28.52_MD589_2_0107_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_mask_cropped.png'

In [34]:
THUMBNAIL_DATA_DIR

'/shared/CSHL_data_processed'

In [36]:
metadata_cache['anchor_fn']

{'MD585': 'MD585-N47-2015.07.16-22.50.52_MD585_3_0141',
 'MD589': 'MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091',
 'MD590': 'MD590-N47-2015.09.12-05.32.06_MD590_2_0140',
 'MD591': 'MD591-N57-2015.09.21-17.13.29_MD591_1_0169',
 'MD592': 'MD592-IHC59-2015.08.26-00.06.25_MD592_2_0176',
 'MD593': 'MD593-IHC54-2015.08.25-17.40.32_MD593_2_0161',
 'MD594': 'MD594-IHC58-2015.08.26-18.48.50_MD594_1_0172',
 'MD595': 'MD595-IHC48-2015.09.15-18.38.44_MD595_1_0142',
 'MD598': 'MD598-N42-2015.09.29-19.26.12_MD598_2_0125',
 'MD599': 'MD599-IHC40-2015.10.03-06.04.31_MD599_2_0119',
 'MD602': 'MD602-N49-2015.12.01-18.41.46_MD602_2_0146',
 'MD603': 'MD603-N60-2015.12.03-23.57.05_MD603_2_0179',
 'MD635': 'MD635-F63-2016.05.19-08.39.03_MD635_2_0188',
 'MD642': 'MD642-F53-2017.01.14-12.23.43_MD642_1_0157',
 'MD652': 'MD652-F45-2016.12.17-05.56.31_MD652_2_0134',
 'MD653': 'MD653-F64-2016.12.21-12.03.55_MD653_2_0191',
 'MD657': 'MD657-F44-2017.02.18-06.06.27_MD657_1_0130',
 'MD658': 'MD658-N58-2017.03.31-19.5

In [39]:
metadata_cache['sections_to_filenames']['MD589']

{1: 'MD589-N1-2015.07.30-16.19.59_MD589_1_0001',
 2: 'MD589-IHC1-2015.07.30-21.30.07_MD589_1_0001',
 3: 'MD589-N1-2015.07.30-16.19.59_MD589_2_0002',
 4: 'MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002',
 5: 'MD589-N1-2015.07.30-16.19.59_MD589_3_0003',
 6: 'MD589-IHC1-2015.07.30-21.30.07_MD589_3_0003',
 7: 'MD589-N2-2015.07.30-16.22.26_MD589_1_0004',
 8: 'MD589-IHC2-2015.07.30-21.32.04_MD589_1_0004',
 9: 'MD589-N2-2015.07.30-16.22.26_MD589_2_0005',
 10: 'MD589-IHC2-2015.07.30-21.32.04_MD589_2_0005',
 11: 'MD589-N2-2015.07.30-16.22.26_MD589_3_0006',
 12: 'MD589-IHC2-2015.07.30-21.32.04_MD589_3_0006',
 13: 'MD589-N3-2015.07.30-16.25.24_MD589_1_0007',
 14: 'MD589-IHC3-2015.07.30-21.34.34_MD589_1_0007',
 15: 'MD589-N3-2015.07.30-16.25.24_MD589_2_0008',
 16: 'MD589-IHC3-2015.07.30-21.34.34_MD589_2_0008',
 17: 'MD589-N3-2015.07.30-16.25.24_MD589_3_0009',
 18: 'MD589-IHC3-2015.07.30-21.34.34_MD589_3_0009',
 19: 'MD589-N4-2015.07.30-16.28.25_MD589_1_0010',
 20: 'MD589-IHC4-2015.07.30-21.37.21_MD58